In [1]:
%%html
<style>
table {float:left}
</style>

In [2]:
import csv
from pyhpeimc.auth import *
from pyhpeimc.plat.groups import *

# Input IMC Credentials


Create new auth object which contains the URL, username, password to access the IMC server

In [6]:
auth = IMCAuth("http://", "10.101.0.203", "8080", "admin", "admin")

# Working with Single Custom Views

In this section, we will look at the various functions designed to work with individual custom views. 

## Create Print View List Helper Function

In [7]:
def print_views():
    views_list = get_custom_views(url=auth.url, auth=auth.creds)
    print ("There are a total of " + str(len(views_list)) + " views currently")
    for view in views_list:
        print (view['name'])
    print (json.dumps(views_list[0], indent = 4))

## Display the current views

This section will count the total number of existing views as well as display the first view returned.

Notice the 'upLevelSymbolId" is **3** for this custom view. The three designates that this is a level one custom view, meaning that it's at the root of the custom view tree.

In [8]:
print_views()

There are a total of 1 views currently
My Network View
{
    "name": "My Network View",
    "statusImgSrc": "res/icon_custom_view_critical_16x16.png",
    "statusDesc": "Critical",
    "upLevelSymbolId": "3",
    "symbolId": "11",
    "runStatus": "5"
}


![Default Custom Views](./images/customviews1.png "Default Custom Views")

## Create New Custom View

In this step, we will create two custom views **Canada** and **Alberta** using the create_custom_views() function.

For this example, we will create 
- the **Canada** view which is a L1 custom view
- the **Alberta** custom view which will be a child of the **Canada** Custom view.
- the **Calgary** custom view which will be a child of the **Alberta** Custom View

|Name| Upperview |
|:-----|-----|
|Canada|     |
|Alberta| Canada |
|Calgary | Alberta |

In [9]:
create_custom_views(auth=auth.creds, url=auth.url, name="Canada")
create_custom_views(auth=auth.creds, url=auth.url, name="Alberta",upperview='Canada')
create_custom_views(auth=auth.creds, url=auth.url, name="Calgary",upperview='Alberta')


print_views()

{ "name": "Canada",
         "upLevelSymbolId" : ""}
{ "name": "Alberta",
        "upLevelSymbolId" : "1480"}
{ "name": "Calgary",
        "upLevelSymbolId" : "1481"}
There are a total of 4 views currently
Alberta
Calgary
Canada
My Network View
{
    "statusDesc": "Unmanaged",
    "name": "Alberta",
    "statusImgSrc": "res/icon_custom_view_unmanaged_16x16.png",
    "runStatus": "-1",
    "symbolId": "1481"
}


![Manual Custom Views](./images/customviews2.png "Manual Custom Views")

### Canada View

Notice that the **upLevelSymbolId** for this view is the same as the default **My Network View** shown above. We can see that this is a L1 custom view. 

In [8]:
get_custom_views(url=auth.url, auth=auth.creds, name="Canada")

[{'name': 'Canada',
  'runStatus': '-1',
  'statusDesc': 'Unmanaged',
  'statusImgSrc': 'res/icon_custom_view_unmanaged_16x16.png',
  'symbolId': '1312',
  'upLevelSymbolId': '3'}]

### Alberta View

Notice that **upLevelSymbolId** for this view is equal to the **symbolId** of the **Canada** view shown in the previous example. we can see that this is a child of the Canada Custom View.

In [9]:
get_custom_views(url=auth.url, auth=auth.creds, name="Alberta")

[{'name': 'Alberta',
  'runStatus': '-1',
  'statusDesc': 'Unmanaged',
  'statusImgSrc': 'res/icon_custom_view_unmanaged_16x16.png',
  'symbolId': '1313',
  'upLevelSymbolId': '1312'}]

### Calgary View

Notice that **upLevelSymbolId** for this view is equal to the **symbolId** of the **Alberta** view shown in the previous example. we can see that this is a child of the Canada Custom View.

In [10]:
get_custom_views(url=auth.url, auth=auth.creds, name="Calgary")

[{'name': 'Calgary',
  'runStatus': '-1',
  'statusDesc': 'Unmanaged',
  'statusImgSrc': 'res/icon_custom_view_unmanaged_16x16.png',
  'symbolId': '1314',
  'upLevelSymbolId': '1313'}]

## Deleting Custom Views

In this section we will delete individual custom views.

In [10]:
delete_custom_view(url=auth.url, auth=auth.creds, name='Canada')
delete_custom_view(url=auth.url, auth=auth.creds, name='Alberta')
delete_custom_view(url=auth.url, auth=auth.creds, name='Calgary')
print_views()

There are a total of 1 views currently
My Network View
{
    "name": "My Network View",
    "statusImgSrc": "res/icon_custom_view_critical_16x16.png",
    "statusDesc": "Critical",
    "upLevelSymbolId": "3",
    "symbolId": "11",
    "runStatus": "5"
}


![Deleted Custom Views](./images/customviews4.png "Deleted Custom Views")

# Display Contents of Custom Views.csv

We have prepared a CSV file which contains the new custom views that we wish to create. This can contain both parent and child custom views. 

| name | upperview |
| ------ | ----------- |
| Branches   |   |
| Branch1 | Branches |
| Branch2    | Branches |
| WAN      |            |

In [11]:
with open('custom_views.csv') as f:
    s = f.read()
    print (s)

name,upperview
Branches,
Branch1,Branches
Branch2,Branches
WAN,



# Create Import Custom Views Function

Here we create a new function which will take a CSV file shown above as an input to the create_custom_views function

In [12]:
def import_custom_views(filename):
    """
    Function which takes in a csv files as input to the create_custom_views function from the pyhpimc python module
    available at https://github.com/HPNetworking/HP-Intelligent-Management-Center
    :param filename: user-defined filename which contains two column named "name" and "upperview" as input into the
    create_custom_views function from the pyhpimc module.
    :return: returns output of the create_custom_vies function (str) for each item in the CSV file.
    """
    with open(filename) as csvfile:
        # decodes file as csv as a python dictionary
        reader = csv.DictReader(csvfile)
        for view in reader:
            # loads each row of the CSV as a JSON string
            name = view['name']
            upperview = view['upperview']
            if len(upperview) is 0:
                upperview = None
            create_custom_views(auth=auth.creds, url=auth.url,name=name,upperview=upperview)

# Import Custom Views from CSV File

In [13]:
import_custom_views('custom_views.csv')

{ "name": "Branches",
         "upLevelSymbolId" : ""}
{ "name": "Branch1",
        "upLevelSymbolId" : "1483"}
{ "name": "Branch2",
        "upLevelSymbolId" : "1483"}
{ "name": "WAN",
         "upLevelSymbolId" : ""}


In [14]:
views_list = get_custom_views(url=auth.url, auth=auth.creds)
print_views()
print ("There are a total of " + str(len(views_list)) + " views currently")

There are a total of 5 views currently
Branch1
Branch2
Branches
My Network View
WAN
{
    "runStatus": "-1",
    "name": "Branch1",
    "statusImgSrc": "res/icon_custom_view_unmanaged_16x16.png",
    "symbolId": "1306",
    "statusDesc": "Unmanaged"
}
There are a total of 5 views currently


![Imported Custom Views](./images/customviews5.png "Imported Custom Views")

In [5]:
views_list = ['Branches', 'Branch1', 'Branch2', "WAN"]

In [6]:
for view in views_list:
    delete_custom_view(url=auth.url, auth=auth.creds, name=view)

In [7]:
views_list = get_custom_views(url=auth.url, auth=auth.creds)
print ("There are a total of " + str(len(views_list)) + " views currently")

There are a total of 1 views currently


![Deleted Custom Views](./images/customviews4.png "Deleted Custom Views")